In [1]:
import numpy as np

from pycircstat2 import Circular, load_data

## Descriptive Statistics

### Table of Contents

- [circ_mean](#circular-mean)
- [circ_moment](#circular-moment)
- [circ_dispersion](#circular-dispersion)
- [circ_skewness](#circular-skewness)
- [circ_kurtosis](#circular-kurtosis)
- [circ_median](#circular-median)

### See also

Chapter 26 of Zar (2010) contains many examples and step-by-step guide of how to compute most of circular descriptive statistics. We replicated all those examples and figures in notebook [`B2-Zar-2010`](https://nbviewer.org/github/circstat/pycircstat2/blob/main/examples/B2-Zar-2010.ipynb) with `pycircstats2`.

### Circular mean

`circ_mean(alpha, w)` returns both the circular mean and the mean resultant length, as both quantities are closely related and computed together.

In [2]:
from pycircstat2.descriptive import circ_mean

b11 = load_data("B11", source="fisher")["θ"].values
c11 = Circular(data=b11)

u, r = circ_mean(alpha=c11.alpha)
print(f"μ={np.rad2deg(u).round(2)}, r={r.round(2)}")

μ=3.1, r=0.83


### Circular moment

`circ_moment(alpha, w)` returns the `p`th trigonometric moment and the corresponding mean resultant vector (and some intermediates values if `return_intermediates=True`)

In [3]:
from pycircstat2.descriptive import circ_moment

# first moment == mean

u1, r1, Cbar, Sbar = circ_moment(alpha=c11.alpha, p=1, centered=False, return_intermediates=True)
print(f"μ1={np.rad2deg(u1).round(2)}, r1={r1.round(2)}, ∑cos={Cbar.round(2)}, ∑sin={Sbar.round(2)}")

# second moment

u2, r2, Cbar, Sbar = circ_moment(alpha=c11.alpha, p=2, centered=False, return_intermediates=True)
print(f"μ2={np.rad2deg(u2).round(2)}, r2={r2.round(2)}, ∑cos2={Cbar.round(2)}, ∑sin2={Sbar.round(2)}")

μ1=3.1, r1=0.83, ∑cos=0.83, ∑sin=0.04
μ2=0.64, r2=0.67, ∑cos2=0.67, ∑sin2=0.01


### Circular dispersion

`circ_dispersion` computes the **sample circular dispersion** (eq 2.28, Fisher, 1993):

$$\text{circular dispersion} = \frac{1 - r_{2}}{2 r_{1}^{2}}$$

* NOTE: In Fisher (1993), data were centered (by substrating the mean) before the computation of all summary statistics related to the second circular moments (such as dispersion, skewness and kurtosis), which might results in *incorrect* estimates (see [issue #58](https://github.com/circstat/pycircstat/issues/58) from `pycircstat`). Here, we follows Pewsey, et al. 2014, by setting the default as **NOT** centering the data. 

In [4]:
from pycircstat2.descriptive import circ_dispersion

circ_dispersion(alpha=c11.alpha), (1 - r2)/ (2 * r1 **2)

(0.23986529263377035, 0.23986529263377035)

### Circular skewness

Circular skewness measures how symmetric the data distribution is:

$$\text{circular skewness} = \frac{r_{2}\cos(\mu_{2} - 2\mu_{1})}{(1 - r_{1})^{3/2}}$$

If the resulting value is near 0, then the data distribution is symmetric; if it's relatively large and negative, the data distribution is skewed counterclockwise away from the mean direction; if it's positive, then it's skewed clockwise.


In [5]:
from pycircstat2.descriptive import circ_skewness

circ_skewness(alpha=c11.alpha), (r2 * np.sin(u2 - 2 * u1)) / (1 - r1) ** 1.5


(-0.9235490965405332, -0.9235490965405332)

### Circular kurtosis

Circular kurtosis measures the peakedness of the data distribution:

$$\text{circular kurtosis} = \frac{r_{2}\cos(\mu_{2} - 2\mu_{1}) - r_{1}^{4}}{(1 - r_{1})^{2}}$$


If it's close to 0, the data is near evenly distributed around the circle.

In [6]:
from pycircstat2.descriptive import circ_kurtosis

circ_kurtosis(alpha=c11.alpha), (r2 * np.cos(u2 - 2 * u1) - r**4) / (1 - r1) ** 2


(6.642665308562964, 6.642665308562964)

### Circular median

The median for circular data is not well-defined compared to other summary statistics. 

The simplest way to define the circular sample median is to find the axis that can divide the data into two equal halves. In `pycircstat2`, we call this method `count` because it's implemented as rotating an axis, and count the number of data point in both sides, then returns the axis with the smallest count difference. The axes can be just the angles in the data set, or, it can be predefined, for example, all axes seperated by 1 degree. We chose the first approach as it's more convenient. 

Alternatively, one can define the median as the axis that has the minimum mean deviation (`deviation`; in some literature, it's called Mardia's median):

$$\text{mean deviation} = \pi - \frac{1}{n}\sum_{1}^{n}|\pi - |\theta_{i} - \theta||$$

Again, there are two approches: 1. we can choose the axis from existing data points, or 2. from a predefined grids.

For grouped data, we use the method of Mardia (1972).

Both `count` and `deviation` would potentially identify multiple axes that satisfy the condition. In those case, we followed Otieno & Anderson-Cook (2003) by simply taking the circular mean of **ALL** potential medians. But the user can also choose to return all potential medians instead.



In [7]:
from pycircstat2.descriptive import circ_median

# example with multiple ties 
d = np.deg2rad(np.array([0, 45, 45, 135,135, 180]))

# return average: (45 + 135) / 2 = 90
print(np.rad2deg(circ_median(alpha=d, return_average=True)))
# return all potential medians
print(np.rad2deg(circ_median(alpha=d, return_average=False)))

90.0
[ 45.  45. 135. 135.]


## TODO

### Circular standard deviation

`circ_std()`

### Confidence interval for circular mean

`circ_mean_ci()`

### Confidence interval for circular median

`circ_median_ci()`

In [8]:
%load_ext watermark
%watermark --time --date --timezone --updated --python --iversions --watermark -p pycircstat2

Last updated: 2023-01-31 16:59:02CET

Python implementation: CPython
Python version       : 3.10.6
IPython version      : 8.5.0

pycircstat2: 0.1.0

sys  : 3.10.6 | packaged by conda-forge | (main, Aug 22 2022, 20:43:44) [Clang 13.0.1 ]
numpy: 1.23.5

Watermark: 2.3.1

